In [1]:
import argparse
from functools import partial
import os
import json
import random

os.environ['TRANSFORMERS_CACHE'] = "/data/hf/"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import bitsandbytes as bnb
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, Trainer, TrainingArguments, BitsAndBytesConfig, \
    DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets.arrow_dataset import Dataset

%load_ext autoreload
%autoreload 2

## Define

In [2]:
def load_model(model_name, bnb_config):
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)

    # Needed for LLaMA tokenizer
    tokenizer.pad_token = tokenizer.eos_token
    special_tokens = {"additional_special_tokens": ["<START_Q>", "<END_Q>", "<START_A>", "<END_A>", "<DIM>", "<MET>", "<ASC>"]}
    tokenizer.add_tokens(special_tokens)
    model.resize_token_embeddings(len(tokenizer))
    return model, tokenizer

def create_zeta_prompt(sample: dict):
    formatted_prompt = "<START_Q>" + sample["question"] + "<END_Q>" + "<START_A>" + sample["tok_str"] + "<END_A>"
    return {"text": formatted_prompt}

def split_and_save_dataset(input_file, train_file, test_file, test_percentage=0.25):
    # Load the dataset from the input file
    zeta_raw = []
    with open(input_file, "r") as f:
        for line in f:
            zeta_raw.append(json.loads(line))

    # Shuffle the dataset to randomize the order of samples
    random.shuffle(zeta_raw)

    # Determine the index to split the dataset (test_percentage for test set)
    split_index = int(test_percentage * len(zeta_raw))

    # Split the dataset into train and test sets
    train_set = zeta_raw[split_index:]
    test_set = zeta_raw[:split_index]

    # Save the train and test sets to separate files
    with open(train_file, "w") as f:
        for sample in train_set:
            f.write(json.dumps(sample) + "\n")

    with open(test_file, "w") as f:
        for sample in test_set:
            f.write(json.dumps(sample) + "\n")

# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length


def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )


# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed, dataset: Dataset):
    """Format & tokenize it so it is ready for training
    :param tokenizer (AutoTokenizer): Model Tokenizer
    :param max_length (int): Maximum number of tokens to emit from tokenizer
    """
    
    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_zeta_prompt)#, batched=True)
    
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        # remove_columns=['question', 'output', 'text']
    )

    # Filter out samples that have input_ids exceeding max_length
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)
    
    # Shuffle dataset
    dataset = dataset.shuffle(seed=seed)

    return dataset


def create_bnb_config():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    return bnb_config

def create_peft_config(modules):
    """
    Create Parameter-Efficient Fine-Tuning config for your model
    :param modules: Names of the modules to apply Lora to
    """
    config = LoraConfig(
        r=16,  # dimension of the updated matrices
        lora_alpha=64,  # parameter for scaling
        target_modules=modules,
        modules_to_save= ["embed_tokens", "lm_head"],
        lora_dropout=0.1,  # dropout probability for layers
        bias="lora_only",
        task_type="CAUSAL_LM",
    )

    return config

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:  # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)


def print_trainable_parameters(model, use_4bit=False):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        num_params = param.numel()
        # if using DS Zero 3 and the weights are initialized empty
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel

        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params
    if use_4bit:
        trainable_params /= 2
    print(
        f"all params: {all_param:,d} || trainable params: {trainable_params:,d} || trainable%: {100 * trainable_params / all_param}"
    )

## Dataset

In [3]:
# Declare the dataset paths 
rel_path = "../data/aug_4838_20230803/"

input_file = rel_path + "augmented.jsonl"
train_file = rel_path + "train.jsonl"
test_file = rel_path + "test.jsonl"

# Split the dataset into train and test
split_and_save_dataset(input_file, train_file, test_file, test_percentage=0.15)

# Load the dataset in the HuggingFace Dataset format
zeta_train = load_dataset(rel_path, split="train")
zeta_test = load_dataset(rel_path, split="test")

print(f"Train set size: {len(zeta_train)}")
print(f"Test set size: {len(zeta_test)}")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Found cached dataset json (/home/drose/.cache/huggingface/datasets/json/aug_4838_20230803-d0b5314f74ccdd50/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


Dataset json downloaded and prepared to /home/drose/.cache/huggingface/datasets/json/aug_4838_20230803-d0b5314f74ccdd50/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.
Train set size: 1961
Test set size: 345


## Load Cloud Model and Resize

In [4]:
model_name = "meta-llama/Llama-2-7b-hf"
local_model_path = "resized/llama-2-7b-hf-4tokens"
output_dir = "trained/peft_finetuned"

# delete files in both resizes and trained dirs
!rm -rf {local_model_path}
!rm -rf {output_dir}

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)

# Needed for LLaMA tokenizer
tokenizer.pad_token = tokenizer.eos_token
special_tokens = {"additional_special_tokens": ["<START_Q>", "<END_Q>", "<START_A>", "<END_A>"]}
tokenizer.add_special_tokens(special_tokens)
model.resize_token_embeddings(len(tokenizer))

# Save to disk
model.save_pretrained(local_model_path)
tokenizer.save_pretrained(local_model_path)

del model
del tokenizer
torch.cuda.empty_cache()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/drose/miniconda3/envs/dolly2/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


## Load Resized Local Model

In [6]:
bnb_config = create_bnb_config()

# model, tokenizer = load_model(model_name, bnb_config)
model = AutoModelForCausalLM.from_pretrained(
        local_model_path,
        quantization_config=bnb_config,
        device_map="auto",
    )
tokenizer = AutoTokenizer.from_pretrained(local_model_path)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## Preprocess

In [7]:
max_length = get_max_length(model)

train_ds = preprocess_dataset(tokenizer, max_length, 0, zeta_train)
eval_ds = preprocess_dataset(tokenizer, max_length, 0, zeta_test)

print(f"Train set size after preprocessing: {len(train_ds):,}")
print(f"Test set size after preprocessing: {len(eval_ds):,}")

Found max lenth: 4096
Preprocessing dataset...


Map:   0%|          | 0/1961 [00:00<?, ? examples/s]

Map:   0%|          | 0/1961 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1961 [00:00<?, ? examples/s]

Preprocessing dataset...


Map:   0%|          | 0/345 [00:00<?, ? examples/s]

Map:   0%|          | 0/345 [00:00<?, ? examples/s]

Filter:   0%|          | 0/345 [00:00<?, ? examples/s]

Train set size after preprocessing: 1,961
Test set size after preprocessing: 345


## Train

In [8]:
# Apply preprocessing to the model to prepare it by
# 1 - Enabling gradient checkpointing to reduce memory usage during fine-tuning
model.gradient_checkpointing_enable()

# 2 - Using the prepare_model_for_kbit_training method from PEFT
model = prepare_model_for_kbit_training(model)

# Get lora module names
modules = find_all_linear_names(model)

# Create PEFT config for these modules and wrap the model to PEFT
peft_config = create_peft_config(modules)
model = get_peft_model(model, peft_config)

# Training parameters
trainer = Trainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    args=TrainingArguments(
        per_device_train_batch_size=8,
        gradient_accumulation_steps=1,
        per_device_eval_batch_size=64,
        warmup_steps=2,
        num_train_epochs=1,
        # max_steps=20,
        learning_rate=2e-6,
        fp16=True,
        logging_steps=1,
        output_dir=output_dir,
        optim="paged_adamw_8bit",
        evaluation_strategy="steps",
        eval_steps=100,
        push_to_hub=False,
        save_strategy='no',
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False
do_train = True
if do_train:
    train_result = trainer.train()
    metrics = train_result.metrics
    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()
    print(metrics)


# Saving model
print("Saving last checkpoint of the model...")
os.makedirs(output_dir, exist_ok=True)
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

del model
del trainer
del tokenizer
torch.cuda.empty_cache()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: drose (zeta-genai). Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
100,3.034100,2.889304
200,2.847500,2.752087
300,2.387400,2.620203
400,2.657000,2.497869
500,2.382100,2.385683
600,2.090300,2.283309
700,2.188100,2.194868
800,2.201400,2.115142
900,2.190500,2.043813
1000,2.107600,1.980610


***** train metrics *****
  epoch                    =        1.0
  total_flos               =  2367450GF
  train_loss               =     2.1273
  train_runtime            = 0:18:17.83
  train_samples_per_second =      1.786
  train_steps_per_second   =      1.786
{'train_runtime': 1097.8365, 'train_samples_per_second': 1.786, 'train_steps_per_second': 1.786, 'total_flos': 2542030547853312.0, 'train_loss': 2.127295382955377, 'epoch': 1.0}
Saving last checkpoint of the model...


## Export

In [9]:
import os

os.environ['TRANSFORMERS_CACHE'] = "/data/hf/"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from peft import AutoPeftModelForCausalLM
import torch
from transformers import AutoTokenizer


output_dir = "trained/peft_finetuned"
output_merged_dir = "merged/test"

# Load model
model = AutoPeftModelForCausalLM.from_pretrained(
    output_dir, 
    device_map="auto", 
    torch_dtype=torch.bfloat16,
)
model = model.merge_and_unload()

# Save model
os.makedirs(output_merged_dir, exist_ok=True)
model.save_pretrained(
    output_merged_dir,
    safe_serialization=True,
    push_to_hub=True,
    repo_id="cipher982/report_builder"
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(output_dir)

# Save tokenizer
tokenizer.save_pretrained(
    output_merged_dir, 
    push_to_hub=True, 
    repo_id="cipher982/report_builder"
)

!rm -rf {local_model_path}
!rm -rf {output_dir}


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
